In [ ]:
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import GridSearchCV
from scipy.stats import gaussian_kde

#Load your dataset
file_path = "Model_training_data.csv"
# Load the data
df = pd.read_csv(file_path)
#Select feature columns
X_kmers = df.iloc[:, 2:86]  
X_domains = df.iloc[:, 86:86 + 1288] 
X_total_chromosome_length = df[['Total_Chromosome_Length']]  
X_plasmid_length = df[['Plasmid_Length']] 
# Combine all feature columns
X = pd.concat([X_kmers, X_domains, X_total_chromosome_length, X_plasmid_length], axis=1)
# target variable: Log1p_PIRACopyNumber
y = df['Log1p_PIRACopyNumber'] 
# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)
#Perform RandomizedSearchCV for hyperparameter tuning
param_distributions = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=500, num=30)],
    'max_features': ['sqrt', 'log2', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'max_depth': [int(x) for x in np.linspace(10, 300, num=30)],
    'min_samples_split': [2, 5, 10, 15, 20, 30, 40, 50],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10, 20, 30],
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_distributions,
                                   n_iter=100, cv=5, verbose=1, random_state=42, n_jobs=-1)

# Fit the RandomizedSearchCV to the data
random_search.fit(X_train, y_train)
#extract the best parameters:
best_params_random_search = random_search.best_params_
# Get the best parameters
print("Best Parameters from Randomized Search:", random_search.best_params_)
# Our Best Parameters from Randomized Search for example 
#Best Parameters from Randomized Search: {'n_estimators': 113, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 0.4, 'max_depth': 290, 'bootstrap': True}
# If you run this code on your own, use the second parameter grid below. It should be tuned based on the values printed above.
# These values don't need to be identical, as they depend on the random splits from your specific run, but they should not cause significant deviations in the final output.

# Now perform extensive grid search around the best parameters from random search 
param_grid2 = {
    'n_estimators': [best_params_random_search['n_estimators'] - 25, best_params_random_search['n_estimators'], best_params_random_search['n_estimators'] + 25],
    'max_features': [best_params_random_search['max_features'], 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'max_depth': [best_params_random_search['max_depth'] - 10, best_params_random_search['max_depth'], best_params_random_search['max_depth'] + 10],
    'min_samples_split': [best_params_random_search['min_samples_split'] - 1, best_params_random_search['min_samples_split'], best_params_random_search['min_samples_split'] + 1],
    'min_samples_leaf': [1, 2, 3], 
    'bootstrap': [best_params_random_search['bootstrap']]  
}
# Initialize GridSearchCV 
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid2, cv=5, n_jobs=-1, verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and score from GridSearchCV
best_grid_params = grid_search.best_params_
best_grid_score = grid_search.best_score_

print(f"Best parameters from GridSearchCV: {best_grid_params}")
print(f"Best score from GridSearchCV: {best_grid_score}")
# Initialize the final Random Forest model with the best parameters from GridSearchCV
rf_model_final = RandomForestRegressor(
    n_estimators=best_grid_params['n_estimators'],
    max_depth=best_grid_params['max_depth'],
    max_features=best_grid_params['max_features'],
    min_samples_split=best_grid_params['min_samples_split'],
    min_samples_leaf=best_grid_params['min_samples_leaf'],
    bootstrap=best_grid_params['bootstrap'],
    random_state=42
)

# Fit the final model on the entire training data
rf_model_final.fit(X_train, y_train)
# Predict on the test set
y_pred = rf_model_final.predict(X_test)
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
spearman_corr, _ = spearmanr(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")
print(f"Spearman's Rank Correlation: {spearman_corr}")

# Save actual vs predicted values
results_df = pd.DataFrame({
    'SampleID': X_test.index if hasattr(X_test, 'index') else range(len(X_test)),
    'Actual': y_test.values if hasattr(y_test, 'values') else y_test,
    'Predicted': y_pred
})

results_csv_path = "all_features_actual_vs_predicted_final.csv"
results_df.to_csv(results_csv_path, index=False)

# Save evaluation metrics summary
metrics_df = pd.DataFrame([{
    'Feature_Set': 'All Features',
    'R2': r2,
    'MAE': mae,
    'MSE': mse,
    'Spearman': spearman_corr
}])

metrics_csv_path = "all_features_evaluation_metrics_final.csv"
metrics_df.to_csv(metrics_csv_path, index=False)

print(f"Saved actual vs predicted values to {results_csv_path}")
print(f"Saved evaluation metrics summary to {metrics_csv_path}")

# Save the model
model_path = "/rf_model_final.pkl"
joblib.dump(rf_model_final, model_path)
print(f"Model saved at: {model_path}")

# Back-transform the targets and predictions (inverse of log1p)
y_test_orig = np.expm1(y_test)  # Back-transform actual values
y_pred_orig = np.expm1(y_pred)

# Back-transform actual values and save to the actual vs predicted final file

df=pd.read_csv('all_features_actual_vs_predicted_final.csv')
x0=df.iloc[:,3]
y0=df.iloc[:,4]
# Assume x0, y0 already defined

XX1 = np.log10(x0)
YY1 = np.log10(y0)
# Filter out NaN and inf values
valid_mask = ~np.isnan(XX1) & ~np.isinf(XX1) & ~np.isnan(YY1) & ~np.isinf(YY1)
XX1 = XX1[valid_mask]
YY1 = YY1[valid_mask]

# Calculate the density
xy = np.vstack([XX1, YY1])
z = gaussian_kde(xy)(xy)

# Sort the points by density so that the densest points are plotted last
idx = z.argsort()
XX1, YY1, z = XX1.iloc[idx], YY1.iloc[idx], z[idx]

plt.figure(figsize=(3, 2))
# Use density values for coloring
sc = plt.scatter(XX1, YY1, c=z, marker='o', s=5, edgecolor=None, cmap='Reds', vmin=0, vmax=0.8)

# Add colorbar for point density
cb = plt.colorbar(sc)
cb.set_label('Point Density', fontsize=8)

# Perform linear regression using statsmodels
X = sm.add_constant(XX1)
model = sm.OLS(YY1, X).fit()

# Generate points for the regression line
x_fit = np.linspace(XX1.min(), XX1.max(), 100)
X_fit = sm.add_constant(x_fit)
y_fit = model.predict(X_fit)

# Calculate the confidence interval
prediction = model.get_prediction(X_fit)
ci = prediction.conf_int()

# Plot the regression line
plt.plot(x_fit, y_fit, color='red', linewidth=1)

# Plot the confidence interval
plt.fill_between(x_fit, ci[:, 0], ci[:, 1], color='red', alpha=0.2)

# Calculate and print Spearman correlation
corr, p_value = spearmanr(XX1, YY1)
print(f"Spearman's rho: {corr:.3f}, p-value: {p_value:.3g}")

plt.xlabel('Actual PCN (log10)')
plt.ylabel('Predicted PCN (log10)')
plt.savefig('fig1d.pdf', bbox_inches='tight')
plt.show()
